In [22]:
from pyannote.audio import Pipeline
from transformers import pipeline as transcribe_pipeline
import torchaudio
from huggingface_hub import login
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import torch
from pydub import AudioSegment
import librosa

from config import HF_TOKEN

In [3]:
login(HF_TOKEN)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /Users/michael/.cache/huggingface/token
Login successful


In [4]:
# Initialize pyannote pipeline for speaker diarization
print("loading pyannote pipeline")
diarization_pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1")

# Initialize Whisper model for ASR (automatic speech recognition)
print("loading whisper model")
whisper_model = transcribe_pipeline("automatic-speech-recognition", model="openai/whisper-base")

# Path to your audio file
audio_file = "09-07-2017 audio_6 APCNV-2016-565.mp3"

# Perform diarization
print("performing diarization")
diarization = diarization_pipeline(audio_file)


loading pyannote pipeline


loading whisper model


performing diarization


In [29]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
# model_id = "openai/whisper-large-v3"
model_id = "openai/whisper-base"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)


In [30]:

# Load the audio file
print("loading audio file")
# waveform, sample_rate = torchaudio.load(audio_file)

# audio = AudioSegment.from_mp3(audio_file)
waveform, sample_rate = librosa.load(audio_file, sr=16000)  # Load and resample to 16kHz



# Process each segment and transcribe
transcriptions = []
print("processing segments")

previous_speaker = None
current_speaker_text = ""
for segment in diarization.itertracks(yield_label=True):
    start_time = segment[0].start
    end_time = segment[0].end
    speaker_label = segment[2]
    
    # Extract the segment from the waveform
    # start_sample = int(start_time * sample_rate)
    # end_sample = int(end_time * sample_rate)
    # segment_waveform = waveform[:, start_sample:end_sample]
    # segment_waveform = waveform[start_time:end_time]
        # Convert time to sample indices
    start_sample = int(start_time * sample_rate)
    end_sample = int(end_time * sample_rate)
    
    # Ensure indices are within bounds
    start_sample = max(0, start_sample)
    end_sample = min(len(waveform), end_sample)
    
    # Extract the segment from the waveform
    segment_waveform = waveform[start_sample:end_sample]


    transcription = pipe(segment_waveform)
    # transcription = result["text"]

    # Convert the segment to text using Whisper
    # transcription = whisper_model(segment_waveform.squeeze().numpy(), return_timestamps=False)
    current_speaker = speaker_label.split('_')[-1]
    
    # Handle the first speaker
    if previous_speaker is None:
        current_speaker_text = transcription['text']


    elif current_speaker != previous_speaker:
        # Append the previous speaker's text if it exists
        if current_speaker_text:
            transcriptions.append(f"Speaker {previous_speaker.split('_')[-1]}: {current_speaker_text}")
        current_speaker_text = transcription['text']
        print(f"Speaker {speaker_label.split('_')[-1]}: {transcription['text']}")
    else:
        current_speaker_text += " " + transcription['text']

    previous_speaker = current_speaker

    # Handle the last speaker after the loop
if current_speaker_text:
    transcriptions.append(f"Speaker {current_speaker}: {current_speaker_text}")


# Print the transcriptions
for transcription in transcriptions:
    print(transcription)

loading audio file
processing segments
Speaker 01:  Hello, everyone. My name is Lucerito Martinez. I'm the project planner assigned to the case. The project consists of the demolition of an existing single family home and the subsequent construction of a three-story eight-unit apartment building with a semi-subterranean garage. It will provide a total of 15 parking spaces. The requested entitlements include a zone change from R1-1 to R3-1 to construct the 8-unit apartment building and also include say 15-foot building line removal created by ordinance 1-29661. The recommendation is to approve the categorical exemption class 32, the zone change from R1-1 to TQR3-1.
Speaker 09:  That's the report. Do we have any questions from the Commission for the Department on the report?
Speaker 04:  No questions.
Speaker 09:  Okay.
Speaker 04:  you
Speaker 09:  Thank you.
Speaker 05:  you


There was an error while processing timestamps, we haven't found a timestamp as last token. Was WhisperTimeStampLogitsProcessor used?


Speaker 00:  you
Speaker 04:  I'm sending this one.
Speaker 00:  you
Speaker 09:  It's a general comment request.
Speaker 05:  If there's a comment card...
Speaker 09:  i don't have i don't have any cards for against the side
Speaker 04:  comment on item six.
Speaker 07:  Yeah.
Speaker 04:  Are we?
Speaker 07:  Yeah, this is where NACA before.
Speaker 09:  I'm just happy.
Speaker 03:  Good.
Speaker 09:  By the way
Speaker 08:  yes i'd like to know what is the name of the puppet well bully bull bully bull but the ball everybody goes by must
Speaker 09:  But I'm going to be back.
Speaker 08:  by their name.
Speaker 09:  Thank you.
Speaker 08:  Mr. Puppet. Well, no, but he's known as Mr. Puppet. Okay. Please go ahead. You got one minute.
Speaker 09:  you
Speaker 06:  So on my comment card, they put down general comments about item six. So on the corner on the top left, is that right?
Speaker 08:  Web.
Speaker 06:  There's an item number that's listed.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:38                                                                                   │
│                                                                                                  │
│   35 │   segment_waveform = waveform[start_sample:end_sample]                                    │
│   36 │                                                                                           │
│   37 │                                                                                           │
│ ❱ 38 │   transcription = pipe(segment_waveform)                                                  │
│   39 │   # transcription = result["text"]                                                        │
│   40 │                                                                                           │
│   41 │   # Convert the segment to text using Whisper                                             │
│                                                                                                  │
│ /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/pip │
│ elines/automatic_speech_recognition.py:272 in __call__                                           │
│                                                                                                  │
│   269 │   │   │   │   │   │   "there", "timestamps": (1.0, 1.5)}]`. The original full text can   │
│   270 │   │   │   │   │   │   `"".join(chunk["text"] for chunk in output["chunks"])`.            │
│   271 │   │   """                                                                                │
│ ❱ 272 │   │   return super().__call__(inputs, **kwargs)                                          │
│   273 │                                                                                          │
│   274 │   def _sanitize_parameters(                                                              │
│   275 │   │   self,                                                                              │
│                                                                                                  │
│ /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/pip │
│ elines/base.py:1112 in __call__                                                                  │
│                                                                                                  │
│   1109 │   │   elif is_iterable:                                                                 │
│   1110 │   │   │   return self.iterate(inputs, preprocess_params, forward_params, postprocess_p  │
│   1111 │   │   elif self.framework == "pt" and isinstance(self, ChunkPipeline):                  │
│ ❱ 1112 │   │   │   return next(                                                                  │
│   1113 │   │   │   │   iter(                                                                     │
│   1114 │   │   │   │   │   self.get_iterator(                                                    │
│   1115 │   │   │   │   │   │   [inputs], num_workers, batch_size, preprocess_params, forward_pa  │
│                                                                                                  │
│ /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/pip │
│ elines/pt_utils.py:124 in __next__                                                               │
│                                                                                                  │
│   121 │   │   │   return self.loader_batch_item()                                                │
│   122 │   │                                                                                      │
│   123 │   │   # We're out of items within a batch                                                │
│ ❱ 124 │   │   item = next(self.iterator)                   